In [1]:
import ranking
import salvar_resultados as sr
import funcoes_st as fst

import pandas as pd
import numpy as np
import string

import time

In [2]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_val = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_val.append(df_val)
    lista_df_teste.append(df_teste)

# Usando os modelos ST

In [3]:
flag_st = True
if flag_st == True:

    nome_modelo, apelido_modelo = fst.setar_modelo(0)
    #tam = 51

    dir_metricas = f"Dados/Resultados/Ranqueado/ST-{apelido_modelo}"

    for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

        '''df_teste = df_teste[:tam]
        df_treino = df_teste[:tam]
        df_val = df_teste[:tam]'''

        df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')
        
        inicio_tempo = time.time()
        embedding = fst.pipeline_st(df_treino, df_val, df_concat, nome_modelo)
        resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)

        final_tempo = time.time()

        # adicionando o tempo que demorou para o algoritmo rodar
        ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

        # calcular acuracia_k e match rank
        indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

        #salvar métricas
        sr.salvar_resultado(nome, df_matches, dir_metricas)
        sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

Epoch: 100%|██████████| 3/3 [42:56<00:00, 858.93s/it]
c:\Users\llvs2\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
    a

NameError: name 'a' is not defined

# Resultados

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

In [ ]:
def retornar_resultados(metodo):
    
    lista_df_resultado = []
    for arquivo in arquivos:

        df_r = pd.read_csv(f"Dados/Resultados/Ranqueado/{metodo}/{arquivo}_métricas.csv")
        lista_df_resultado.append(df_r)

    return lista_df_resultado

## ST-dbm-v1

In [ ]:
metodo = "ST-dbm-v1"
lista_resultado_dbm_v1 = retornar_resultados(metodo)

### ale_1_1

In [ ]:
lista_resultado_dbm_v1[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

### ale_5_1

In [ ]:
lista_resultado_dbm_v1[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

### hn_1_1

In [ ]:
lista_resultado_dbm_v1[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

### hn_5_1

In [ ]:
lista_resultado_dbm_v1[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()